### NX-421: Neural signals and signal processing
# Miniproject 1 - Variant 3

### Camille Dorster, Toufan Kashaev, Johan Bordet

### Dataset Description
The dataset used is a snippet issued from the Human Connectome Project. In particular, this task was adapted from a previously developed one by Buckner and colleagues (Buckner et al. 2011 [2]). Participants are shown visual cues asking them to either tap their left or right fingers, or squeeze their left or right toes, or move their tongue, the goal being to map motor areas. Each block of a movement type lasted ~12 seconds (10 movements), and is preceded by a ~3 second cue. In each of the two runs, there are 13 blocks, with 2 of tongue movements, 4 of hand movements (2 right and 2 left), and 4 of foot movements (2 right and 2 left). In addition, there are 3 15-second fixation blocks per run.

### Importing the relevant libraries

In [1]:
import sys
import os

#####################
# Import of utils.py functions
#####################
notebook_dir = os.path.abspath("")
parent_dir = os.path.abspath(os.path.join(notebook_dir, '..'))
sys.path.append(parent_dir)
sys.path.append('.')
from utils import loadFSL, FSLeyesServer, mkdir_no_exist, interactive_MCQ

####################
# DIPY_HOME should be set prior to import of dipy to make sure all downloads point to the right folder
####################
os.environ["DIPY_HOME"] = "/home/jovyan/data"

#############################
# Loading fsl and freesurfer within Neurodesk
# You can find the list of available other modules by clicking on the "Softwares" tab on the left
#############################
import lmod
await lmod.purge(force=True)
await lmod.load('fsl/6.0.7.4')
await lmod.load('freesurfer/7.4.1')
await lmod.list()

####################
# Setup FSL path
####################
loadFSL()

import fsl.wrappers
from fsl.wrappers import fslmaths

import mne_nirs
import nilearn
from nilearn.datasets import fetch_development_fmri
from nilearn.image import load_img, new_img_like, concat_imgs
from nilearn.masking import compute_epi_mask, apply_mask, unmask

import mne
import mne_nirs
import dipy
from dipy.data import fetch_bundles_2_subjects, read_bundles_2_subjects
import xml.etree.ElementTree as ET
import os.path as op
import nibabel as nib
import glob

import ants

import openneuro
from mne.datasets import sample
from mne_bids import BIDSPath, read_raw_bids, print_dir_tree, make_report


# Useful imports to define the direct download function below
import requests
import urllib.request
from tqdm import tqdm


# FSL function wrappers which we will call from python directly
from fsl.wrappers import fast, bet
from fsl.wrappers.misc import fslroi
from fsl.wrappers import flirt

# General purpose imports to handle paths, files etc
import glob
import pandas as pd
import numpy as np
import json
import subprocess

# Utility function used to load json from file name
def get_json_from_file(fname):
    f = open(fname)
    data = json.load(f)
    f.close()
    return data

%gui wx

# ###################
# Preparing the paths and structure of the data in folders
###################

sample_path = "/home/jovyan/data"
dataset_id = 'mydataset'
subject_id = '101410' 
subject_dir = 'sub-{}'.format(subject_id)

bids_root = op.join(sample_path, dataset_id)
deriv_root = op.join(bids_root, 'derivatives')
preproc_root = op.join(bids_root, 'derivatives','preprocessed_data')
preproc_subject_path = op.join(preproc_root, subject_dir)
preproc_anat_path = op.join(preproc_root, subject_dir, 'anat')
preproc_func_path = op.join(preproc_root, subject_dir, 'func')
preproc_fmap_path = op.join(preproc_root, subject_dir, 'fmap')
LR_tfMRI_data_path = op.join(bids_root, 'sub-101410', 'func', 'sub-{}_tfMRI_MOTOR_LR'.format(subject_id))
RL_tfMRI_data_path = op.join(bids_root, 'sub-101410', 'func', 'sub-{}_tfMRI_MOTOR_RL'.format(subject_id))
concat_tfMRI_data_path = op.join(preproc_func_path, 'sub-{}_task-motor_concat_scaled.nii.gz'.format(subject_id))
gaussian_filtered_path = op.join(preproc_func_path, 'sub-{}_task-motor_concat_bbr'.format(subject_id))
path_moco_data = op.join(preproc_func_path, 'sub-{}_tfMRI_MOTOR_LR_moco'.format(subject_id))

###################
# Create folders relevant for preprocessing.
###################
mkdir_no_exist(sample_path)
mkdir_no_exist(bids_root)
mkdir_no_exist(deriv_root)
mkdir_no_exist(preproc_root)
mkdir_no_exist(preproc_subject_path)
mkdir_no_exist(preproc_anat_path)
mkdir_no_exist(preproc_func_path)
mkdir_no_exist(preproc_fmap_path)

Gtk-Message: 17:04:10.865: Failed to load module "canberra-gtk-module"


In [2]:
################
# Start FSLeyes
################
fsleyesDisplay = FSLeyesServer()
fsleyesDisplay.show()

17:04:14: Debug: Adding duplicate image handler for 'Windows bitmap file'
17:04:14: Debug: Adding duplicate animation handler for '1' type
17:04:14: Debug: Adding duplicate animation handler for '2' type
17:04:14: Debug: Adding duplicate image handler for 'Windows bitmap file'
17:04:14: Debug: Adding duplicate animation handler for '1' type
17:04:14: Debug: Adding duplicate animation handler for '2' type

(ipykernel_launcher.py:8303): Gtk-CRITICAL **: 17:04:15.248: gtk_window_resize: assertion 'height > 0' failed


In [6]:
print_dir_tree(bids_root, max_depth=4)

|mydataset/
|--- dataset_description.json
|--- derivatives/
|------ preprocessed_data/
|--------- sub-101410/
|------------ anat/
|------------ fmap/
|------------ func/
|--- sub-101410/
|------ anat/
|--------- sub-101410_T1w.nii.gz
|--------- .ipynb_checkpoints/
|------ func/
|--------- events_LR.csv
|--------- events_RL.csv
|--------- task-motor_bold.json
|--------- tfMRI_MOTOR_LR.nii
|--------- tfMRI_MOTOR_RL.nii


## Step 1 : skull stripping

### Applying BET to get the betted brain mask

In [7]:
resulting_mask_path = op.join(preproc_anat_path, 'sub-{}_T1w_mask'.format(subject_id))
anatomical_path = op.join(bids_root, subject_dir, 'anat', 'sub-{}_T1w.nii.gz'.format(subject_id))
betted_brain_path = op.join(preproc_anat_path, 'sub-{}_T1w'.format(subject_id))

os.system('bet {} {} -m {}'.format(anatomical_path, betted_brain_path, '-R')) # Robust parameter used

print("Done with BET.")

Done with BET.


In [8]:
# Display the mask
fsleyesDisplay.load(resulting_mask_path)

### Applying the betted brain mask to the original brain

In [5]:
anatomical_path = op.join(bids_root, subject_dir, 'anat', 'sub-{}_T1w.nii.gz'.format(subject_id)) # The original brain
betted_brain_path = op.join(preproc_anat_path, 'sub-{}_T1w.nii.gz'.format(subject_id)) # The brain without skull is in the derivatives folder
resulting_mask_path = op.join(preproc_anat_path, 'sub-{}_T1w_mask'.format(subject_id)) # The mask to use

os.system('fslmaths {} -mas {} {}'.format(anatomical_path, resulting_mask_path, betted_brain_path))

print("Mask applied.")

Mask applied.


In [10]:
# Display the betted brain
fsleyesDisplay.resetOverlays()
fsleyesDisplay.load(betted_brain_path)


(ipykernel_launcher.py:2471): Gdk-WARNING **: 14:19:49.359: gdkdrawable-x11.c:952 drawable is not a pixmap or window


## Step 2 : tissue segmentation

In [11]:
fast_target = betted_brain_path

[os.remove(f) for f in glob.glob(op.join(preproc_anat_path, '*fast*'))] # Just to clean the directory in between runs of the cell
segmentation_path = op.join(preproc_anat_path, 'sub-101410_T1w_fast')
fast(imgs=[fast_target], out=segmentation_path, n_classes=3)

{}

In [6]:
fast_target = betted_brain_path
# Display the segmented brain
fsleyesDisplay.resetOverlays()
fsleyesDisplay.load(betted_brain_path)
fsleyesDisplay.load(glob.glob(op.join(preproc_anat_path,'*pve_0*'))[0])
fsleyesDisplay.load(glob.glob(op.join(preproc_anat_path,'*pve_1*'))[0])
fsleyesDisplay.load(glob.glob(op.join(preproc_anat_path,'*pve_2*'))[0])
fsleyesDisplay.displayCtx.getOpts(fsleyesDisplay.overlayList[1]).cmap = 'Red'
fsleyesDisplay.displayCtx.getOpts(fsleyesDisplay.overlayList[2]).cmap = 'Green'
fsleyesDisplay.displayCtx.getOpts(fsleyesDisplay.overlayList[3]).cmap = 'Blue'

## Step 3 : Variance normalization and concatenation of the runs

In [15]:
# $ python -m pip install fmriprep-docke
run_files = [
    LR_tfMRI_data_path + '.nii',
    RL_tfMRI_data_path + '.nii',
]

# Mask to select only the brain
runs = [load_img(r) for r in run_files]
mask = compute_epi_mask(runs[0])
for run in runs[1:]:
    mask = new_img_like(mask, (mask.get_fdata()>0) & (compute_epi_mask(run).get_fdata()>0))

scaled_runs = []
for run in runs:
    # Mask to get (T, V)
    X = apply_mask(run, mask)                # shape (T, V)
    # # Mean & demean of the whole run
    # mu = X.mean()
    # X = X - mu
    # Variance of the run & rescale to unit variance
    # ddof=1 for sample variance; add tiny eps for numerical safety
    var = X.var(ddof=1)
    X = X / np.sqrt(var + 1e-8)
    # Put back to 4D image with original affine/header
    scaled_img = unmask(X, mask)
    scaled_runs.append(scaled_img)

# Concatenate timewise AFTER scaling
concat_img = concat_imgs(scaled_runs)  # -> sub-101410_task-motor_concat_scaled.nii.gz (in memory)
nib.save(concat_img, concat_tfMRI_data_path)


In [16]:
fsleyesDisplay.resetOverlays()
fsleyesDisplay.load(concat_tfMRI_data_path)

## Step 4 : motion correction

In [17]:
from fsl.wrappers import mcflirt

mcflirt(infile=concat_tfMRI_data_path,o=path_moco_data, plots=True, report=True, dof=6, mats=True)

Processed data will be saved as /home/jovyan/data/mydataset/derivatives/preprocessed_data/sub-101410/func/sub-101410_tfMRI_MOTOR_LR_moco

McFLIRT v 2.0 - FMRI motion correction

Reading time series... 
first iteration - 8mm scaling, set tolerance
Rescaling reference volume [284] to 8 mm pixels
Registering volumes ... [285][286][287][288][289][290][291][292][293][294][295][296][297][298][299][300][301][302][303][304][305][306][307][308][309][310][311][312][313][314][315][316][317][318][319][320][321][322][323][324][325][326][327][328][329][330][331][332][333][334][335][336][337][338][339][340][341][342][343][344][345][346][347][348][349][350][351][352][353][354][355][356][357][358][359][360][361][362][363][364][365][366][367][368][369][370][371][372][373][374][375][376][377][378][379][380][381][382][383][384][385][386][387][388][389][390][391][392][393][394][395][396][397][398][399][400][401][402][403][404][405][406][407][408][409][410][411][412][413][414][415][416][417][418][419][420][

refnum = 284
Original_refvol = -1


Registering volumes ... [285][286][287][288][289][290][291][292][293][294][295][296][297][298][299][300][301][302][303][304][305][306][307][308][309][310][311][312][313][314][315][316][317][318][319][320][321][322][323][324][325][326][327][328][329][330][331][332][333][334][335][336][337][338][339][340][341][342][343][344][345][346][347][348][349][350][351][352][353][354][355][356][357][358][359][360][361][362][363][364][365][366][367][368][369][370][371][372][373][374][375][376][377][378][379][380][381][382][383][384][385][386][387][388][389][390][391][392][393][394][395][396][397][398][399][400][401][402][403][404][405][406][407][408][409][410][411][412][413][414][415][416][417][418][419][420][421][422][423][424][425][426][427][428][429][430][431][432][433][434][435][436][437][438][439][440][441][442][443][444][445][446][447][448][449][450][451][452][453][454][455][456][457][458][459][460][461][462][463][464][465][466][467][468][469][470][471][472][473][474][475][476][477][478][479][

{}

In [3]:
fsleyesDisplay.resetOverlays()
fsleyesDisplay.load(concat_tfMRI_data_path)
fsleyesDisplay.load(path_moco_data)


(ipykernel_launcher.py:10160): Gdk-WARNING **: 15:30:54.252: gdkdrawable-x11.c:952 drawable is not a pixmap or window


## Step 5:Guaussian smoothing

In [5]:
cmd = 'fslmaths {} -s {} {}_smoothed-6mm'.format(concat_tfMRI_data_path, 6/2.3548, gaussian_filtered_path)
subprocess.run(['fslmaths',concat_tfMRI_data_path, '-s', str(6/2.3548), '{}_smoothed-6mm'.format(gaussian_filtered_path)])


CompletedProcess(args=['fslmaths', '/home/jovyan/data/mydataset/derivatives/preprocessed_data/sub-101410/func/sub-101410_task-motor_concat_scaled.nii.gz', '-s', '2.547987090198743', '/home/jovyan/data/mydataset/derivatives/preprocessed_data/sub-101410/func/sub-101410_task-motor_concat_bbr_smoothed-6mm'], returncode=0)

In [11]:
fsleyesDisplay.resetOverlays()
#fsleyesDisplay.load(gaussian_filtered_path)
fsleyesDisplay.load(gaussian_filtered_path + '_smoothed-6mm')

OSError: Expected 2050773088 bytes, got 60846472 bytes from object
 - could the file be damaged?

Traceback (most recent call last):
  File "/opt/conda/lib/python3.11/site-packages/fsleyes/displaycontext/displaycontext.py", line 524, in getDisplay
    display = self.__displays[overlay]
              ~~~~~~~~~~~~~~~^^^^^^^^^
KeyError: Image(sub-101410_task-motor_concat_bbr_smoothed-6mm, /data/mydataset/derivatives/preprocessed_data/sub-101410/func/sub-101410_task-motor_concat_bbr_smoothed-6mm.nii.gz)

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/lib/python3.11/site-packages/fsleyes/displaycontext/displaycontext.py", line 524, in getDisplay
    display = self.__displays[overlay]
              ~~~~~~~~~~~~~~~^^^^^^^^^
KeyError: Image(sub-101410_task-motor_concat_bbr_smoothed-6mm, /data/mydataset/derivatives/preprocessed_data/sub-101410/func/sub-101410_task-motor_concat_bbr_smoothed-6mm.nii.gz)

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/op

Let's review one last time the different steps you've studied and which FSL tool(s) you used to do it:
<table>
    <tr><th style='text-align:justify;'>Data type</th><th style='text-align:justify;'>Step name </th><th style='text-align:justify;'>Details of the step</th><th style='text-align:justify;'>FSL tool </th></tr>
    <tr><th>Anatomical</th><td></td><td></td></tr>
    <tr><td></td><td style='text-align:justify;'>Skull stripping</td><td style='text-align:justify;'>Removing skull and surrounding tissues to keep only the brain</td><td style='text-align:justify;'>BET</td></tr>
    <tr><td></td><td style='text-align:justify;'>Segmentation</td><td style='text-align:justify;'>Segmenting brain tissues based on their contrasts</td><td style='text-align:justify;'>FAST</td></tr>
    <tr><td></td><td style='text-align:justify;'>Normalization</td><td style='text-align:justify;'>Mapping participant's brain to a reference brain, making its orientation and scale match so that comparison across participants become feasible.</td><td style='text-align:justify;'>FLIRT + FNIRT (from last week), or ANTs</td></tr>
    <tr><th>Functional</th><th></th><th></th></tr>
    <tr><td></td><td style='text-align:left;'>First few volumes removal</td><td style='text-align:justify;'>Removing volumes for which the B0 field is still not stable and that could contaminate all our data if left unchecked.</td><td style='text-align:justify;'>fslroi</td></tr>
    <tr><td></td><td style='text-align:left;'>Motion correction</td><td style='text-align:justify;'>Realignment of fMRI volumes to a common reference - typically one volume or the average of the volumes - to correct for inter-volume motion. The extracted motion parameters can be used for subsequent analysis (see GLM next week!)</td><td style='text-align:justify;'>MCFLIRT (which is one suboption of FLIRT in fact)</td></tr>
    <tr><td></td><td style='text-align:left;'>Coregistration to anatomical</td><td style='text-align:justify;'>Putting the functional volumes in anatomical space</td><td style='text-align:justify;'>FLIRT (epi_reg being a specialized instance)</td></tr>
    <tr><td></td><td style='text-align:left;'>Smoothing</td><td style='text-align:justify;'>Allowing a bit of lee-way in the voxel's values to account for the imperfection of the registration</td><td style='text-align:justify;'>fslmath with smoothing operation</td></tr>
</table>

## Step 6: GLM 

In [6]:

# Load both event files
events1 = pd.read_csv(op.join(bids_root, 'sub-101410', 'func', 'events_LR.csv'))
events2 = pd.read_csv(op.join(bids_root, 'sub-101410', 'func', 'events_RL.csv'))

# Combine the two DataFrames
events_combined = pd.concat([events1, events2], ignore_index=True)
events_combined.to_csv(op.join(bids_root, 'sub-101410', 'func','combined_events.csv'), index=False)
events = pd.read_csv(op.join(bids_root, 'sub-101410', 'func','combined_events.csv'))
events

,onset,duration,condition
0,0.0,8.0,NaN
1,8.0,3.0,cue
2,11.0,12.0,rh
3,23.0,3.0,cue
4,26.0,12.0,lf
5,38.0,3.0,cue
6,41.0,12.0,t
7,53.0,3.0,cue
8,56.0,12.0,rf
9,68.0,3.0,cue


In [ ]:
from nilearn.glm.first_level import make_first_level_design_matrix, FirstLevelModel

# Specify what sort of GLM we want (nature of the noise, repetition time of the data and other parameters)
fmri_glm = FirstLevelModel(t_r=0.72,
                           noise_model='ar1',
                           standardize=False,
                           hrf_model='spm',
                           drift_model=None,
                           high_pass=.01)

# Fit the model to our design and data
fmri_glm = fmri_glm.fit(concat_tfMRI_data_path, events)

/opt/conda/lib/python3.11/site-packages/nilearn/glm/first_level/experimental_paradigm.py:129: UserWarning: 'trial_type' column not found in the given events data.
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/nilearn/glm/first_level/experimental_paradigm.py:167: UserWarning: The following unexpected columns in events data will be ignored: condition
  warnings.warn(
/opt/conda/lib/python3.11/site-packages/nilearn/glm/first_level/experimental_paradigm.py:198: UserWarning: Duplicated events were detected. Amplitudes of these events will be summed. You might want to verify your inputs.
  warnings.warn(


In [ ]:
from nilearn.plotting import plot_design_matrix
plot_design_matrix(fmri_glm.design_matrices_[0])
plt.show()